In [1]:
import os
import argparse
import torch
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn, optim
from data import HierDataModule
from data import infer_preprocess
from ERDE import ERDE_sample
from model import HierClassifier
from transformers import AutoTokenizer
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from collections import defaultdict, Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from model import HierClassifier
from ERDE import ERDE_chunk
import xml.dom.minidom
import string

/opt/conda/envs/Han/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 注意
version_55前后正态分布sigma不同需要切换 55后为1

In [2]:
clf = HierClassifier.load_from_checkpoint(
    # without time factor"lightning_logs/version_20/checkpoints/epoch=5-step=732.ckpt"
    #with time factor"lightning_logs/version_34/checkpoints/epoch=6-step=854.ckpt"
    #"lightning_logs/version_41/checkpoints/epoch=2-step=366.ckpt"
    # basic Tanh"lightning_logs/version_45/checkpoints/epoch=5-step=732.ckpt"
    #sigmoid"lightning_logs/version_46/checkpoints/epoch=4-step=610.ckpt"
    #relu "lightning_logs/version_47/checkpoints/epoch=4-step=610.ckpt"
    #basic factor"lightning_logs/version_48/checkpoints/epoch=8-step=1098.ckpt"
    #basic factor + norm "lightning_logs/version_51/checkpoints/epoch=4-step=610.ckpt" #second-with bug
    # "lightning_logs/version_53/checkpoints/epoch=4-step=610.ckpt"
    # "lightning_logs/version_55/checkpoints/epoch=3-step=488.ckpt"
    #"lightning_logs/version_57/checkpoints/epoch=4-step=610.ckpt"   #bug fixed
    #"lightning_logs/version_58/checkpoints/epoch=4-step=610.ckpt"   #sota basic schedule
    #"lightning_logs/version_96/checkpoints/epoch=3-step=488.ckpt"    #fusion
    #"lightning_logs/version_187/checkpoints/epoch=2-step=366.ckpt"
    #"lightning_logs/version_190/checkpoints/epoch=6-step=854.ckpt"
    "lightning_logs/version_200/checkpoints/epoch=4-step=610.ckpt"
)
clf.eval()
tokenizer = AutoTokenizer.from_pretrained(clf.model_type)
max_len = clf.hparams.max_len
max_posts = 16
clf.cuda()
None

/opt/conda/envs/Han/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/opt/conda/envs/Han/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
with open("miniLM_L6_embs.pkl", "rb") as f:
    data = pickle.load(f)

train_posts = data["train_posts"]
train_mappings = data["train_mappings"]
train_tags = data["train_labels"]
train_embs = data["train_embs"]
train_timepoints = data["train_timepoints"]
test_posts = data["test_posts"]
test_mappings = data["test_mappings"]
test_tags = data["test_labels"]
test_embs = data["test_embs"]
test_timepoints = data["test_timepoints"]
test_analyze = data["test_analyzes"]

In [4]:
sbert = SentenceTransformer('/mnt/proj/paraphrase-MiniLM-L6-v2').cuda()

In [5]:
depression_texts = [
    "I feel depressed.",
    "I am diagnosed with depression.",
    "I am treating my depression."
]
questionaire_single = [
    "I feel sad.",
    "I am discouraged about my future.",
    "I always fail.",
    "I don't get pleasure from things.",
    "I feel quite guilty.",
    "I expected to be punished.",
    "I am disappointed in myself.",
    "I always criticize myself for my faults.",
    "I have thoughts of killing myself.",
    "I always cry.",
    "I am hard to stay still.",
    "It's hard to get interested in things.",
    "I have trouble making decisions.",
    "I feel worthless.",
    "I don't have energy to do things.",
    "I have changes in my sleeping pattern.",
    "I am always irritable.",
    "I have changes in my appetite.",
    "I feel hard to concentrate on things.",
    "I am too tired to do things.",
    "I have lost my interest in sex."
]

template_embeddings = sbert.encode(depression_texts+questionaire_single)
template_embeddings.shape

(24, 384)

In [6]:
time_dim = 768
assert time_dim%24==0,'time dim should be '
hour_span = time_dim//24
minutes_span = 60//hour_span
half_time_dim = time_dim//2
# Normal distribution
sigma = 1
#sigma = 0.03 # befor version_55
x = np.arange(-time_dim/2, time_dim/2, 1)
y = np.multiply(np.power(np.sqrt(2 * np.pi) * sigma, -1), np.exp(-np.power(x, 2) / 2 * sigma ** 2))
norm = torch.from_numpy(y).to(torch.float32)/16

In [7]:
def trans_timeindexs(timepoints):
    hour_index = timepoints[:,0]
    time_index = hour_index
    return time_index.long()
def norm_emb(timepoints:torch.tensor)->torch.tensor:
    #norm_x = torch.arange(-self.time_dim/2,self.time_dim/2,1)
    #raise Exception(norm_x) #[-xxx,...,xxx-1]
        
    #index = hour * hour_span + minute//minutes_span
    # raise Exception("shape,dtype", timepoints.shape,timepoints.dtype) #('shape', (1131, 2)) int64
    indexs = timepoints[:,0] * hour_span + torch.floor(timepoints[:,0] / minutes_span).to(torch.int)
    indexs = indexs.to(torch.int)
    #raise Exception(indexs.max(),indexs.min())
    embs = torch.zeros(0,time_dim)
    for index in indexs:
        timeline = norm
        if index<half_time_dim:
            cut = half_time_dim-index
            left = norm[:cut]
            right = norm[cut:]
            timeline = torch.cat((right,left),dim=0)
        elif index> half_time_dim:
            cut = index-half_time_dim
            left = norm[:time_dim-cut]
            right = norm[time_dim-cut:]
            timeline = torch.cat((right,left),dim=0)
        embs = torch.cat((embs,timeline.unsqueeze(0)),dim=0)
    return embs

In [8]:
sample_pred_probas = []
user_basis = []
num_updates_users = []
num_posts_users = []
for mappings in tqdm(test_mappings, total=len(test_mappings)):
    user_posts = [test_posts[i] for i in mappings[::-1]]
    timepoints = [test_timepoints[i] for i in mappings]
    analyzes = [test_analyze[i] for i in mappings]
    #raise Exception(analyzes.shape)
    timepoints_ = np.array(timepoints)
    timepoints_ = torch.from_numpy(timepoints_)
    timepoints_emb = norm_emb(timepoints_)
    timeindexs = trans_timeindexs(timepoints_)
    embs = torch.from_numpy(test_embs[mappings])
    #raise Exception(timepoints_emb.shape,embs.shape)
    pred_probas = []
    posts_bank = []
    analyze_bank = []
    embedding_bank = None
    scores_bank = []
    basis_bank = []
    num_updates = 0
    for pid, new_post in enumerate(user_posts):
        # new_post = ""
        # new_emb = sbert.encode(new_post).reshape(1, -1)
        new_emb = test_embs[mappings[pid]].reshape(1, -1)
        new_scores = cosine_similarity(new_emb, template_embeddings)[0]
        best_template_id = new_scores.argmax()
        new_score = new_scores[best_template_id]
        new_analyze = analyzes[pid]
        # take all new posts before capacity is all used
        tpemb_bank = timepoints_emb[:pid,:]
        tpindex_bank = timeindexs[:pid]
        textemb_bank = embs[:pid,:]
        #print(pid)
        if len(posts_bank) < max_posts:
            posts_bank.insert(0, new_post)
            scores_bank.insert(0, new_score)
            basis_bank.insert(0, best_template_id)
            analyze_bank.insert(0, new_analyze)
            #print(analyze_bank,posts_bank)
            batch = infer_preprocess(tokenizer, posts_bank, max_len, tpemb_bank, textemb_bank,tpindex_bank,analyze_bank)
            for k, v in batch.items():
                batch[k] = v.cuda()
            with torch.no_grad():
                logits, attn_score,sc = clf([batch])
            num_updates += 1
            proba = torch.sigmoid(logits).detach().cpu().item()
            pred_probas.append(proba)
            continue
        min_id = np.argmin(scores_bank)
        if new_score >= scores_bank[min_id]:
            del posts_bank[min_id]
            del scores_bank[min_id]
            del basis_bank[min_id]
            del analyze_bank[min_id]
            posts_bank.insert(0, new_post)
            scores_bank.insert(0, new_score)
            basis_bank.insert(0, best_template_id)
            analyze_bank.insert(0, new_analyze)
            # make prediction
            batch = infer_preprocess(tokenizer, posts_bank, max_len, tpemb_bank, textemb_bank,tpindex_bank,analyze_bank)
            for k, v in batch.items():
                batch[k] = v.cuda()
            with torch.no_grad():
                logits, attn_score,sc = clf([batch])
            num_updates += 1
            proba = torch.sigmoid(logits).detach().cpu().item()
            pred_probas.append(proba)
            # TODO stop if meet condition
        else:
            pred_probas.append(pred_probas[-1])
            # do nothing, save time
            pass
    sample_pred_probas.append(pred_probas)
    num_updates_users.append(num_updates)
    num_posts_users.append(len(user_posts))
    user_basis.append(basis_bank)
len(sample_pred_probas)

100%|██████████| 401/401 [41:53<00:00,  6.27s/it] 


401

In [9]:
num_updates_users = pd.Series(num_updates_users)
num_posts_users = pd.Series(num_posts_users)

In [10]:
pd.DataFrame({
    "num_user_posts": num_posts_users.describe(),
    "num_infers": num_updates_users.describe(),
    "infer_portion": (num_updates_users / num_posts_users).describe()
})

,num_user_posts,num_infers,infer_portion
count,401.000000,401.000000,401.000000
mean,589.453865,61.376559,0.325146
std,589.965998,25.972973,0.299691
min,10.000000,10.000000,0.036500
25%,86.000000,39.000000,0.080808
50%,330.000000,62.000000,0.191617
75%,1074.000000,82.000000,0.506849
max,2000.000000,124.000000,1.000000


In [11]:
# portion of actual model inferences
(num_updates_users.sum() / num_posts_users.sum())

0.10412444843064504

In [12]:
ERDE5 = ERDE_sample(sample_pred_probas, test_tags, threshold=0.5, o=5)
ERDE50 = ERDE_sample(sample_pred_probas, test_tags, threshold=0.5, o=50)
print(ERDE5, ERDE50)

0.08624549975747224 0.06759191783539566
